# AI Technologies 2019

##  Student Project by Kumulil Kiran, Gonzalez Avilés Ruben Octavio, Aerni Florian



## Introduction

A insurance company registered, that the eligibility assessment of applicants varied, depending on which employee assessed it. In order to harmonize the decision and maybe improve the decision finding process, the insurance company decided to implement a computer based system that consists of three parts:

### 1. A decision-based system 
For the decision based system the human experts doing the assesments thus far were interviewed. The findings were formulated into written rules, these rules are the basis for the system.

### 2. A Machine Learning-based system
With the idea to maybe uncover hidden factors, we let a Neural Network (NN) to process data derived from past cases. A first goal is to train the NN, so that it can reproduce a decision when given a test case. In a further step, we try to derive, whether we focus on the right factors or if additional insight can be gained and injected into the decision making.

### 3. The combination of the both systems
Both systems are of relevance, but they might produce variying weightings in assessment. If the insights gained from the NN point in another direction than the decision-based system, we have to look carefully into these results. Further, both systems could produce insights valuable for refining the other tool and highlighting important factors.



## 1. A decision-based System

### Formalizing the knowledge
We first take a step back and have a look in which context this system is supposed to be active:
An Applicant enters their data over a webpage. This data then is translated into the knowledge based system, where it is assessed. The trigger to assess the data will most probably be automated, for simplicity reason we here assume that an employee will manually execute the assessment.

We have to bring the data derived from expert interviews into a format a machine can make us of. As an intermediate step, we translated the text-form results into a decision table:

#### Decision Table Expert Knowledge
https://drive.google.com/file/d/1ZiuXdyh2DM_yBvQFejp_SVModPjLILV9/view?usp=sharing



### The Prolog Program

Starting from the decision table, we started to build a formalized version, namely in Prolog code. To simulate the first steps in the scenario, namely the data entry into the knowledge system, we created two ficitious customers named "Dan" and "Peter". For now the name serves as identifier, in runtime a unique identifier would have to be defined, for example an assigned applicant number.

In this first attempt to formalize the rules as far as possible, there still was an uncertainty left about the relation between eligibility and the risk status. It was first interpreted, that risk status is not the output variable, rather part of the input, and the decision Eligible/Eligible with fee/not Eligible is asked for, thus the first version put out these. For the sake of completeness, this version can be found here:
https://drive.google.com/file/d/15jV8mw57pn7DaAWkugTG31JrCkh4xaDN/view?usp=sharing
These uncertainties later were resolved by clarification through our Expert (Prof. Dr. Holger Wache). The three risk status are synonym with the three eligibility status, thus the risk status is already the output and not considered part of the input. A new and final version of the Prolog Code was developed:

#### Final Protoype Decision-Based System in Prolog
https://drive.google.com/file/d/1HfEQzJ7mw5BnAvqAkpAIWOo0S7C5ySHI/view?usp=sharing


We here briefly provide insight about the functionality of the program sections, in addition to what is commented in the code itself:

- The first section "Facts" is purely the simulation of the data input by the customers. 

- The section "Basic rejection argument" is checking the two basic guidlines for immediate rejection (Applicant not resident in Switzerland, Applicant younger than 21 or older than 70).

- The section "Risk evaluation" is the core section of the program, where an applicant fulfilling basic requirements is acutally assessed. It is all pretty straight forward, checking the conditions described in the decision table. We here would like to point out "not_special_factors()", which is in the negated for for easier understanding - it makes use of the "\+" for negation in prolog. Also, the two last predicates incorporate the inequation 4=5 to reverse when fireing, because we want to return "true" when the risk is medium (see last bullet point for detail)

- A word about the "counting function", which turned out to be a piece de resistance. It took quite some experimenting and research online before we found a working way, making use of two prolog predicates length() and findall(). In our case, we exlusively use the function for counting the number of diseases, thus we hardcoded has_disease() in the counter. The way our counter works is as following: the method takes in a name (unique identifier of records) as well as the empty variable "Count". We pass the name into the findall predicate, which produces a list X, containing all cases of predicates being true with the given Name and any disease ("_" is passed as placeholder). After that, we use the length() method to pass the lenght of the list into the Count variable, which then will be given out to the caller of count(). Et voilà, disease counter.

- To simplify the evaluation, the different checks are summarized in "is_eligible(NAME)". This is the call/query an employee would execute (prolog code: ?-is_eligible(peter).). It puts out the eligibility result, i.e. the risk status. We decided that the result should "true", when the candidate is either eligible or eligible with a risk compensation and therefore only "false" when the candidate is rejected. In any case one of the special factors fires, we provide an additional explenation making use of the "write()" functionality.



## 2. A Machine Learning-based System

## 3. The combination of the both systems

## Insights

## A students perspective